# 🌍 Trajectories of a Freely Released Payload Near Earth: Problem 3

<div style="background-color: #f0f8ff; padding: 15px; border-radius: 10px;">
<h2 style="color: #2E86C1; text-align: center;">🚀 Payload Trajectories: A Comprehensive Analysis</h2>
</div>

---

## 🎯 Task: A Comprehensive Breakdown

We aim to analyze and simulate the trajectories of a payload released near Earth, considering the following objectives:

- **Trajectory Analysis**: Explore the possible trajectories (elliptical, parabolic, hyperbolic) and their conditions.
- **Numerical Computation**: Compute the payload’s path using numerical integration.
- **Orbital Scenarios**: Relate trajectories to real-world space mission scenarios (orbital insertion, reentry, escape).
- **Computational Simulation**: Develop a Python tool to simulate and visualize the payload’s motion, with options to vary initial conditions.

---

## 📜 Trajectory Analysis: Elliptical, Parabolic, and Hyperbolic Paths

The trajectory of a payload is determined by its **total mechanical energy** ($E = KE + PE$), which depends on its initial velocity and position relative to Earth.

### 🌀 Elliptical Trajectory
- **Condition**: Total mechanical energy $E < 0$.
- **Velocity**: Initial velocity $v < v_{\text{escape}}$, where $v_{\text{escape}} = \sqrt{\frac{2GM}{r}}$.
- **Description**: The payload is gravitationally bound to Earth, following a closed elliptical orbit.
- **Application**: Used for satellites in stable orbits (e.g., Low Earth Orbit).

### 📏 Parabolic Trajectory
- **Condition**: Total mechanical energy $E = 0$.
- **Velocity**: Initial velocity $v = v_{\text{escape}}$.
- **Description**: The payload has just enough energy to escape to infinity, with velocity approaching zero at large distances.
- **Application**: Represents the boundary between bound and unbound orbits.

### 🚀 Hyperbolic Trajectory
- **Condition**: Total mechanical energy $E > 0$.
- **Velocity**: Initial velocity $v > v_{\text{escape}}$.
- **Description**: The payload escapes Earth’s gravity with excess energy, moving away with non-zero velocity at infinity.
- **Application**: Used for interplanetary missions (e.g., Voyager probes).

---

## 🧮 Numerical Analysis: Equations of Motion

### Gravitational Force
The payload’s motion is governed by Newton’s Law of Universal Gravitation:

$$
F = -\frac{G M m}{r^2} r_{\text{hat}}
$$

- $G$: Gravitational constant ($6.674 \times 10^{-11} \, \text{m}^3 \, \text{kg}^{-1} \, \text{s}^{-2}$)
- $M$: Mass of Earth ($5.972 \times 10^{24} \, \text{kg}$)
- $m$: Mass of the payload
- $r$: Distance from Earth’s center
- $r_{\text{hat}}$: Unit vector from Earth to payload

Using Newton’s Second Law ($F = m a$), the equation of motion is:

$$
\frac{d^2 r}{dt^2} = -\frac{G M r}{r^3}
$$

### Numerical Integration
We solve this equation using the **Runge-Kutta 4th Order (RK4)** method for better accuracy compared to the Euler method. The system is converted into first-order equations:

$$
\frac{d r}{dt} = v, \quad \frac{d v}{dt} = -\frac{G M r}{r^3}
$$

### Initial Conditions
- **Position**: Start at 500 km altitude ($r_0 = R_{\text{Earth}} + 500 \, \text{km}$).
- **Velocity**: Vary to achieve elliptical, parabolic, and hyperbolic trajectories.

---

## 🌐 Orbital Scenarios: Real-World Applications

### 🛰️ Orbital Insertion
- **Trajectory**: Elliptical.
- **Goal**: Achieve a stable orbit by ensuring $v < v_{\text{escape}}$.
- **Example**: Satellites like the International Space Station (ISS).

### 🔥 Reentry
- **Trajectory**: Elliptical, with perigee in the atmosphere.
- **Goal**: Control the trajectory to manage heat and forces during atmospheric reentry.
- **Example**: Space Shuttle reentry.

### 🌌 Escape
- **Trajectory**: Parabolic or hyperbolic.
- **Goal**: Exceed escape velocity to leave Earth’s gravitational influence.
- **Example**: Interplanetary probes like New Horizons.

---

## 💻 Computational Simulation: Visualizing Payload Motion

We’ll simulate the payload’s motion for three trajectories (elliptical, parabolic, hyperbolic) and visualize them with:
- **Static Plots**: Trajectories, velocity vs. time, and energy analysis.
- **Animations**: Animated GIFs showing the payload’s motion around Earth.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import imageio.v3 as iio
import os

# Constants
G = 6.67430e-11
M = 5.972e24
R_earth = 6.371e6
altitude = 500e3
r0 = R_earth + altitude
v_escape = np.sqrt(2 * G * M / r0)

# Defined trajectory scenarios
scenarios = [
    {'name': 'Elliptical 1', 'v0': 0.78 * v_escape, 'color': 'deepskyblue', 't_max': 30000},
    {'name': 'Elliptical 2', 'v0': 0.87 * v_escape, 'color': 'royalblue', 't_max': 30000},
    {'name': 'Parabolic', 'v0': v_escape, 'color': 'limegreen', 't_max': 32000},
    {'name': 'Hyperbolic', 'v0': 1.25 * v_escape, 'color': 'crimson', 't_max': 33000},
    {'name': 'Reentry Slow', 'v0': 0.99 * v_escape, 'color': 'orange', 't_max': 32000},
    {'name': 'Reentry Sharp', 'v0': 0.97 * v_escape, 'color': 'darkorange', 't_max': 31000},
]

# RK4 integration
def rk4_step(t, y, dt, derivs):
    k1 = derivs(t, y)
    k2 = derivs(t + dt/2, y + dt*k1/2)
    k3 = derivs(t + dt/2, y + dt*k2/2)
    k4 = derivs(t + dt, y + dt*k3)
    return y + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)

def derivs(t, y):
    x, y, vx, vy = y
    r = np.sqrt(x**2 + y**2)
    if r < R_earth:
        return np.array([0, 0, 0, 0])
    ax = -G * M * x / r**3
    ay = -G * M * y / r**3
    return np.array([vx, vy, ax, ay])

def simulate_trajectory(v0, t_max, n_steps=3000):
    dt = t_max / n_steps
    t = np.linspace(0, t_max, n_steps)
    y = np.zeros((n_steps, 4))
    y[0] = [r0, 0, 0, v0]
    for i in range(1, n_steps):
        y[i] = rk4_step(t[i-1], y[i-1], dt, derivs)
        if np.linalg.norm(y[i, :2]) < R_earth:
            y[i:] = y[i]
            break
    return t, y

# Simulate all scenarios
trajectories = []
for s in scenarios:
    t, y = simulate_trajectory(s['v0'], s['t_max'])
    trajectories.append({**s, 't': t, 'y': y})

# Setup plot
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(-3 * R_earth / 1e6, 3 * R_earth / 1e6)
ax.set_ylim(-3 * R_earth / 1e6, 3 * R_earth / 1e6)
ax.set_title("Chaotic Payload Trajectories (Full Turns)")
ax.set_xlabel("X (Mm)")
ax.set_ylabel("Y (Mm)")
ax.set_aspect("equal")

# Earth image or fallback
try:
    earth_img = iio.imread('https://naturalhistory.si.edu/sites/default/files/styles/width_768/public/media/images/BlueMarble-2001-2002.jpg')
    earth_img = np.flipud(earth_img)
    img = OffsetImage(earth_img, zoom=R_earth / (1e6 * earth_img.shape[0]))
    ab = AnnotationBbox(img, (0, 0), frameon=False)
    ax.add_artist(ab)
except:
    earth = plt.Circle((0, 0), R_earth / 1e6, color='cyan', alpha=0.5)
    ax.add_patch(earth)

# Initialize lines
lines, points = [], []
for traj in trajectories:
    line, = ax.plot([], [], color=traj['color'], label=traj['name'])
    point, = ax.plot([], [], 'o', color=traj['color'], markersize=6)
    lines.append(line)
    points.append(point)

ax.legend()

def init():
    for line, point in zip(lines, points):
        line.set_data([], [])
        point.set_data([], [])
    return lines + points

def animate(i):
    for traj, line, point in zip(trajectories, lines, points):
        idx = min(i, len(traj['y']) - 1)
        x = traj['y'][:idx + 1, 0] / 1e6
        y = traj['y'][:idx + 1, 1] / 1e6
        line.set_data(x, y)
        point.set_data([x[-1]], [y[-1]])
    return lines + points

# 💾 Save extended & smooth chaotic animation
ani = FuncAnimation(fig, animate, init_func=init, frames=1000, interval=30, blit=True)
ani.save("chaotic_payload_trajectories.gif", writer="pillow", fps=30)
plt.close()

# ✅ Auto open on Windows
try:
    os.startfile("chaotic_payload_trajectories.gif")
except:
    print("GIF saved as 'chaotic_payload_trajectories.gif'. Please open manually.")


In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import imageio.v3 as iio
import os

# Constants
G = 6.67430e-11
M = 5.972e24
R_earth = 6.371e6
altitude = 500e3
r0 = R_earth + altitude
v_escape = np.sqrt(2 * G * M / r0)

# 🎲 Generate randomized payloads
np.random.seed(42)
num_payloads = 12
scenarios = []
for i in range(num_payloads):
    angle = np.deg2rad(np.random.uniform(0, 360))
    speed_factor = np.random.uniform(0.75, 1.3)
    v0 = speed_factor * v_escape
    vx = v0 * np.cos(angle)
    vy = v0 * np.sin(angle)
    color = np.random.choice(["red", "green", "blue", "orange", "purple", "gold", "cyan", "magenta"])
    scenarios.append({
        'name': f'Payload {i+1}',
        'v0': v0,
        'vx': vx,
        'vy': vy,
        'color': color,
        't_max': 30000  # more time to complete turns
    })

# RK4 integrator
def rk4_step(t, y, dt, derivs):
    k1 = derivs(t, y)
    k2 = derivs(t + dt/2, y + dt*k1/2)
    k3 = derivs(t + dt/2, y + dt*k2/2)
    k4 = derivs(t + dt, y + dt*k3)
    return y + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)

def derivs(t, y):
    x, y, vx, vy = y
    r = np.sqrt(x**2 + y**2)
    if r < R_earth:
        return np.array([0, 0, 0, 0])
    ax = -G * M * x / r**3
    ay = -G * M * y / r**3
    return np.array([vx, vy, ax, ay])

def simulate_payload(vx, vy, t_max, n_steps=3000):
    dt = t_max / n_steps
    t = np.linspace(0, t_max, n_steps)
    y = np.zeros((n_steps, 4))
    y[0] = [r0, 0, vx, vy]
    for i in range(1, n_steps):
        y[i] = rk4_step(t[i-1], y[i-1], dt, derivs)
        if np.linalg.norm(y[i, :2]) < R_earth:
            y[i:] = y[i]
            break
    return t, y

# Simulate all trajectories
trajectories = []
for s in scenarios:
    t, y = simulate_payload(s['vx'], s['vy'], s['t_max'])
    trajectories.append({**s, 't': t, 'y': y})

# Plot setup
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(-3*R_earth/1e6, 3*R_earth/1e6)
ax.set_ylim(-3*R_earth/1e6, 3*R_earth/1e6)
ax.set_title("Orbital Roulette: Long & Fast Payload Chaos")
ax.set_xlabel("X (Mm)")
ax.set_ylabel("Y (Mm)")
ax.set_aspect("equal")

# 🌐 Earth texture or fallback
try:
    earth_img = iio.imread('https://naturalhistory.si.edu/sites/default/files/styles/width_768/public/media/images/BlueMarble-2001-2002.jpg')
    earth_img = np.flipud(earth_img)
    img = OffsetImage(earth_img, zoom=R_earth/(1e6 * earth_img.shape[0]))
    ab = AnnotationBbox(img, (0, 0), frameon=False)
    ax.add_artist(ab)
except:
    earth = plt.Circle((0, 0), R_earth/1e6, color='cyan', alpha=0.5)
    ax.add_patch(earth)

# Animation elements
lines, points = [], []
for traj in trajectories:
    line, = ax.plot([], [], color=traj['color'], label=traj['name'])
    point, = ax.plot([], [], 'o', color=traj['color'], markersize=6)
    lines.append(line)
    points.append(point)

ax.legend(loc='upper right')

def init():
    for line, point in zip(lines, points):
        line.set_data([], [])
        point.set_data([], [])
    return lines + points

def animate(i):
    for traj, line, point in zip(trajectories, lines, points):
        idx = min(i, len(traj['y']) - 1)
        x = traj['y'][:idx+1, 0] / 1e6
        y = traj['y'][:idx+1, 1] / 1e6
        line.set_data(x, y)
        point.set_data([x[-1]], [y[-1]])
    return lines + points

# 💾 Save LONG, FAST animation
ani = FuncAnimation(fig, animate, init_func=init, frames=1000, interval=30, blit=True)
ani.save("orbital_roulette.gif", writer="pillow", fps=30)
plt.close()

# ✅ Open automatically (Windows only)
try:
    os.startfile("orbital_roulette.gif")
except:
    print("GIF saved as 'orbital_roulette.gif'. Please open it manually.")
